<a href="https://colab.research.google.com/github/anderson93/meli-challenge-2019/blob/master/PreProcessing%20and%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://meli-data-challenge.s3.amazonaws.com/train.csv.gz
!wget https://meli-data-challenge.s3.amazonaws.com/test.csv

--2019-09-10 18:11:23--  https://meli-data-challenge.s3.amazonaws.com/train.csv.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.216.136.244
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.216.136.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691651384 (660M) [application/x-gzip]
Saving to: ‘train.csv.gz’

train.csv.gz        100%[===================>] 659.61M  29.4MB/s    in 20s     

2019-09-10 18:11:43 (33.1 MB/s) - ‘train.csv.gz’ saved [691651384/691651384]

--2019-09-10 18:11:45--  https://meli-data-challenge.s3.amazonaws.com/test.csv
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 54.231.81.56
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|54.231.81.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16467436 (16M) [text/csv]
Saving to: ‘test.csv’

tes

In [2]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 9.6MB/s 
     |████████████████████████████████| 655kB 35.3MB/s 
     |████████████████████████████████| 1.0MB 53.6MB/s 
     |████████████████████████████████| 808kB 42.6MB/s 
  Created wheel for regex: filename=regex-2019.8.19-cp36-cp36m-linux_x86_64.whl size=609232 sha256=2aac2d2f6bffd61ee6714f9b9ecd94cb8bc532707c974e5fa36db7689ee95d50
  Stored in directory: /root/.cache/pip/wheels/90/04/07/b5010fb816721eb3d6dd64ed5cc8111ca23f97fdab8619b5be
  Created wheel for sacremoses: filename=sacremoses-0.0.33-cp36-none-any.whl size=833106 sha256=888b2093acdcabe90ca29f1e40844d4026dbaa286be222f9929c81a9e708e855
  Stored in directory: /root/.cache/pip/wheels/70/87/56/e40575cca30d12fee8875d523b8878b7aba866a9f03b2fd983
Successfully built regex sacremoses


In [0]:
import pandas as pd

train_n_test = pd.read_csv('train.csv.gz', compression = 'gzip')
validation = pd.read_csv('test.csv')

In [0]:
import numpy as np

sep = ' '
train_n_test['input'] = pd.Series(map(lambda x: x[0]+sep+x[1].replace(' ',sep), train_n_test[['language','title']].values))

label_to_token = {x:i for i,x in enumerate(pd.unique(train_n_test['category']))}
token_to_label = {i:x for i,x in enumerate(pd.unique(train_n_test['category']))}
train_n_test['label_tokenized'] = train_n_test['category'].apply(lambda x: label_dict[x])

In [0]:
from sklearn.model_selection import train_test_split

X = train_n_test[train_n_test['label_quality']=='reliable']['input'].values
y = train_n_test[train_n_test['label_quality']=='reliable']['label_tokenized'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

In [36]:
train = pd.DataFrame({0:y_train, 1:X_train})
test = pd.DataFrame({0:y_test, 1:X_test})
train.head()

,0,1
0,665,spanish Dvd Portátil V-zon Coby
1,1385,portuguese Depiladora Luz Pulsada - Mlay Ipl O...
2,868,spanish Ultrabook Hp Elitebook 840 G1 I5 500gb...
3,25,portuguese Tapete Sisal Macio Clean Linhas Dia...
4,69,portuguese Leite Neocate Lcp 400g Caixa Com 04...


In [37]:
train_df = pd.DataFrame({
    'id':range(len(train)),
    'label':train[0],
    'alpha':['a']*train.shape[0],
    'text': train[1].replace(r'\n', ' ', regex=True)
})

train_df.head()

,id,label,alpha,text
0,0,665,a,spanish Dvd Portátil V-zon Coby
1,1,1385,a,portuguese Depiladora Luz Pulsada - Mlay Ipl O...
2,2,868,a,spanish Ultrabook Hp Elitebook 840 G1 I5 500gb...
3,3,25,a,portuguese Tapete Sisal Macio Clean Linhas Dia...
4,4,69,a,portuguese Leite Neocate Lcp 400g Caixa Com 04...


In [0]:
dev_df = pd.DataFrame({
    'id':range(len(test)),
    'label':test[0],
    'alpha':['a']*test.shape[0],
    'text': test[1].replace(r'\n', ' ', regex=True)
})

dev_df.head()

train_df.to_csv('train.tsv', sep='\t', index=False, header=False)
dev_df.to_csv('dev.tsv', sep='\t', index=False, header=False)

In [9]:
!wget https://raw.githubusercontent.com/ThilinaRajapakse/pytorch-transformers-classification/master/utils.py

--2019-09-10 18:13:24--  https://raw.githubusercontent.com/ThilinaRajapakse/pytorch-transformers-classification/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12933 (13K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  12.63K  --.-KB/s    in 0s      

2019-09-10 18:13:24 (173 MB/s) - ‘utils.py’ saved [12933/12933]



In [0]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

from pytorch_transformers import AdamW, WarmupLinearSchedule

from utils import (convert_examples_to_features,
                        output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
args = {
    'data_dir': '/',
    'model_type':  'xlm',
    'model_name': 'xlm-mlm-tlm-xnli15-1024',
    'task_name': 'multiclass',
    'output_dir': 'outputs/',
    'cache_dir': 'cache/',
    'do_train': True,
    'do_eval': True,
    'fp16': True,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 8,
    'eval_batch_size': 8,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 1,
    'weight_decay': 0,
    'learning_rate': 4e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': False,
    'save_steps': 2000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': False,
    'reprocess_input_data': True,
    'notes': 'Using MeLi Challenge dataset'
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [63]:
with open('args.json', 'w') as f:
    json.dump(args, f)
    
if os.path.exists(args['output_dir']) and os.listdir(args['output_dir']) and args['do_train'] and not args['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args['output_dir']))
    
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

config = config_class.from_pretrained(args['model_name'], 
                                      num_labels=len([str(x) for x in token_to_label.keys()]), 
                                      finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

model = model_class.from_pretrained(args['model_name'])

INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-mlm-tlm-xnli15-1024-config.json from cache at /root/.cache/torch/pytorch_transformers/a4852a980f0b11e78249cdcc71b5d176d87abcbafa83b669949efa2875430f2e.5c9c090b1e1150f496832947811eff8bb318d046786a96e364b8576f7b4660fb
INFO:pytorch_transformers.modeling_utils:Model config {
  "asm": false,
  "attention_dropout": 0.1,
  "bos_index": 0,
  "causal": false,
  "dropout": 0.1,
  "emb_dim": 1024,
  "embed_init_std": 0.02209708691207961,
  "end_n_top": 5,
  "eos_index": 1,
  "finetuning_task": "multiclass",
  "gelu_activation": true,
  "id2lang": {
    "0": "ar",
    "1": "bg",
    "10": "th",
    "11": "tr",
    "12": "ur",
    "13": "vi",
    "14": "zh",
    "2": "de",
    "3": "el",
    "4": "en",
    "5": "es",
    "6": "fr",
    "7": "hi",
    "8": "ru",
    "9": "sw"
  },
  "init_std": 0.02,
  "is_encoder": true,
  "lang2id": {
    "ar": 0,
    "bg": 1,
    "de": 2,
  

In [64]:
model.to(device);
device

device(type='cuda')

In [67]:
task = args['task_name']

processor = processors[task]()
label_list = processor.get_labels(token_to_label.keys())
num_labels = len(label_list)

NameError: ignored

In [54]:
!ls

args.json  drive		__pycache__  test.csv	   train.tsv
dev.tsv    meli-challenge-2019	sample_data  train.csv.gz  utils.py


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp meli-challenge-2019/* drive/My\ Drive/Projekts/meli-challenge-2019

In [0]:
def load_and_cache_examples(task, tokenizer, evaluate=False, undersample_scale_factor=1):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], f"cached_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
    if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
               
    else:
        logger.info("Creating features from dataset file at %s", args['data_dir'])
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args['data_dir']) if evaluate else processor.get_train_examples(args['data_dir'])
        print(len(examples))
        examples  = [example for example in examples if np.random.rand() < undersample_scale_factor]
        print(len(examples))
        
        features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
            cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
            pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
            process_count=2)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset